In [13]:
import torch
from torch.utils import data
from torch import nn
import pytorch_lightning as L

In [14]:
real_X = torch.normal(0,1,(1000,2))
real_w = torch.tensor([2,-3.4]).reshape(2,1)
real_b = 4.8

real_y = torch.matmul(real_X,real_w) + real_b
real_X.shape, real_y.shape

(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [15]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

In [16]:
class MyNet(L.LightningModule):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.net = nn.Sequential(nn.Linear(2,1))
        self.loss = nn.MSELoss()

    def forward(self, x):
        return self.net(x)
    
    def training_step(self, batch, batch_idx):
        loss, output, y = self.__common_step(batch,batch_idx)
        self.log("train_loss",loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss, output, y = self.__common_step(batch,batch_idx)
        self.log("validation_loss",loss)
        return loss
    
    def test_step(self, batch, batch_idx):
        loss, output, y = self.__common_step(batch,batch_idx)
        self.log("test_loss",loss)
        return loss
    
    def __common_step(self, batch, batch_idx):
        X, y = batch
        output = self(X)
        loss = self.loss(output, y)
        return loss, output, y
    
    def predict_step(self, batch, batch_idx):
        X, y = batch
        output = self(X)
        return output

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(),lr=0.03)


In [17]:
model = MyNet()

trainer = L.Trainer(accelerator='cpu',min_epochs=3,max_epochs=10)
trainer.fit(model,load_array([real_X,real_y],10,True))

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
f:\code\deep-learning\.venv\Lib\site-packages\pytorch_lightning\trainer\setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
f:\code\deep-learning\.venv\Lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name | Type       | Params | Mode 
--------------------------------------------
0 | net  | Sequential | 3      | train
1 | loss | MSELoss    | 0      | train
--------------------------------------------
3         Trainable params
0         Non-trainable params
3         Total params
0.000     Total estimated model params size (MB)
3         Modules in train mode
0  

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [18]:
net = nn.Sequential(nn.Linear(2,1))
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

In [19]:
X, y = next(iter(load_array([real_X,real_y],10,False)))

In [20]:
model(X), y

(tensor([[ 0.5380],
         [ 6.2603],
         [ 7.6602],
         [ 4.1343],
         [ 8.9238],
         [-0.6299],
         [ 8.0655],
         [ 6.6094],
         [11.1337],
         [ 4.5751]], grad_fn=<AddmmBackward0>),
 tensor([[ 0.5380],
         [ 6.2604],
         [ 7.6602],
         [ 4.1343],
         [ 8.9238],
         [-0.6299],
         [ 8.0655],
         [ 6.6094],
         [11.1337],
         [ 4.5751]]))